In [79]:
import faker
import uuid
import psycopg2
from psycopg2.extras import register_uuid

In [80]:
fake = faker.Faker()

In [81]:
users = []

for i in range(2000):
    id = uuid.uuid4()
    username = fake.user_name()
    pwd = fake.password()
    users.append(
        {
            'id': id,
            'username': username,
            'pwd': pwd,
        }
    )

In [82]:
profiles = []

for i in range(len(users)):
    user_id = users[i]['id']
    id = uuid.uuid4()
    users[i]['profile_id'] = id
    email = fake.email()
    phone = fake.phone_number()
    about_me = fake.text(128)
    profiles.append(
        {
            'id': id,
            'email': email,
            'phone': phone,
            'about_me': about_me,
            'user_id': user_id,
        }
    )

In [83]:
import random

In [84]:
apps = []

genres = ['Health', 'Food', 'Business', 'Social media', 'Game', 'Music']

for i in range(500):
    id = uuid.uuid4()
    title = fake.text(64)
    downloads = random.randint(0, 100)
    price = round((random.random()*10), 2)
    recommend = 0 == random.randint(0, 1)
    profit = int(random.random()*1e6)
    genre = genres[random.randint(0, len(genres) - 1)]
    apps.append(
        {
            'id': id, 
            'title': title,
            'downloads': downloads,
            'price': price,
            'recommend': recommend,
            'profit': profit,
            'genre': genre,
        }
    )

In [85]:
import itertools
import pandas as pd

In [87]:
df_users = pd.DataFrame(users)
df_apps = pd.DataFrame(apps)
all_pairs = list(itertools.product(df_apps['id'], df_users['id']))

num_pairs = 600000
random_pairs = random.sample(all_pairs, num_pairs)

In [88]:
comments = []

num_pairs = 500000
pairs_for_comments = random.sample(random_pairs, num_pairs)

for i in range(len(pairs_for_comments)):
    id = uuid.uuid4()
    stars = random.randint(1, 5)
    comment_text = fake.text(128)
    user_id = pairs_for_comments[i][1]
    app_id = pairs_for_comments[i][0]
    comments.append(
        {
            'id': id,
            'stars': stars,
            'comment_text': comment_text,
            'user_id': user_id,
            'app_id': app_id,
        }
    )

In [89]:
from datetime import datetime, timedelta

In [90]:
payments = []

num = 1500000

for i in range(num):
    id = uuid.uuid4()
    cash = random.randint(1, 100)
    start_date = datetime.now() - timedelta(days=10)
    end_date = start_date + timedelta(days=20)
    random_date = start_date + (end_date - start_date) * random.random()
    payment_date = random_date.timestamp()
    pair = random_pairs[random.randint(0, len(random_pairs) - 1)]
    user_id = pair[1]
    app_id = pair[0]
    payments.append(
        {
            'id': id,
            'cash': cash,
            'payment_date': payment_date,
            'user_id': user_id,
            'app_id': app_id,
        }
    )


In [91]:
sessions = []

num = 3000000

for i in range(num):
    id = uuid.uuid4()
    date1 = datetime.now() - timedelta(days=10)
    date2 = start_date + timedelta(days=20)
    random_date = start_date + (end_date - start_date) * random.random()
    start_date = random_date
    end_date = start_date + timedelta(seconds=random.random()*1e5)
    pair = random_pairs[random.randint(0, len(random_pairs) - 1)]
    user_id = pair[1]
    app_id = pair[0]
    sessions.append(
        {
            'id': id,
            'start_date': start_date,
            'end_date': end_date,
            'user_id': user_id,
            'app_id': app_id,
        }
    )


In [92]:
conn = psycopg2.connect(
    dbname="dbes",
    user="EgorChitorkin",
    password="261101",
    host="localhost",
    port="5432"
)
psycopg2.extras.register_uuid(conn)

<psycopg2._psycopg.type 'UUID' at 0x00000168BA271080>

In [32]:
cur = conn.cursor()

for user in users:
    cur.execute('INSERT INTO "user" (id, username, pwd) VALUES (%s, %s, %s)',
                (user['id'], user['username'], user['pwd']))

for profile in profiles:
    cur.execute('INSERT INTO profile (id, email, phone, about_me, user_id) VALUES (%s, %s, %s, %s, %s)',
                (profile['id'], profile['email'], profile['phone'], profile['about_me'], profile['user_id']))

for user in users:
    cur.execute('UPDATE "user" SET profile_id = %s WHERE id = %s',
                (user['profile_id'], user['id']))
    
for app in apps:
    cur.execute('INSERT INTO app (id, title, downloads, price, recommend, profit, genre) VALUES (%s, %s, %s, %s, %s, %s, %s)',
                (app['id'], app['title'], app['downloads'], app['price'], app['recommend'], app['profit'], app['genre']))

for i, j in random_pairs:
    cur.execute('INSERT INTO app_user VALUES (%s, %s)',
                (i, j))

for comment in comments:
    cur.execute('INSERT INTO comment VALUES (%s, %s, %s, %s, %s)',
                (comment['id'], comment['stars'], comment['comment_text'], comment['user_id'], comment['app_id']))
    
for payment in payments:
    cur.execute('INSERT INTO payment VALUES (%s, %s, %s, %s, %s)',
                (payment['id'], payment['cash'], datetime.fromtimestamp(payment['payment_date']), payment['user_id'], payment['app_id']))
    
for session in sessions:
    cur.execute('INSERT INTO session VALUES (%s, %s, %s, %s, %s)',
                (session['id'], session['start_date'], session['end_date'], session['user_id'], session['app_id']))

conn.commit()
cur.close()


In [78]:
conn.close()